In [3]:
%load_ext autoreload
%autoreload
%cd /home/jupyter/MathVGerrmandering_CMAT_2021/
from src import *
# from src.data import *
from src.space import *
from src.mcmc import *

# https://dvr.capitol.texas.gov/
level    = 'cntyvtd'
proposal = 'plans2167'  # for 2010 enacted, use 'planc2100' for US congress, 'plans2100' for TX Senate, or 'planh2100' for TX House
contract = 0
# contract = 'proposal'

# D = Data()
S = Space(level=level, proposal=proposal, contract=contract)
# M = MCMC(gpickle=S.gpickle, nodes=S.tbls['nodes'],
#          max_steps   = 5,
#          random_seed = 13,
#          defect_cap  = 100,
#          pop_deviation_target=10)
# M.run_chain()
# M.post_process()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jupyter/MathVGerrmandering_CMAT_2021
Get crosswalks  ... using existing table ... success!
Get assignments ... using existing table ... success!
Get shapes      ... using existing table ... success!
Get census      ... using existing table ... success!
Get elections   ... using existing table ... success!
Get countries   ... using existing table ... success!
Get proposals   ... using existing table ... success!
Get joined      ... using existing table ... success!
Get tabblock    ... using existing table ... success!
Get bg          ... using existing table ... success!
Get tract       ... using existing table ... success!
Get cntyvtd     ... using existing table ... success!
Get cnty        ... using existing table ... success!
Get all         ... using existing table ... success!
Get proposal    ... processing ... creating proposal table from /home/jupyter/redistricting_data/proposals/TX/202

In [8]:
S.tbls

{'crosswalks': 'cmat-315920.redistricting_data.TX_2020_crosswalks',
 'assignments': 'cmat-315920.redistricting_data.TX_2020_assignments',
 'shapes': 'cmat-315920.redistricting_data.TX_2020_shapes',
 'census': 'cmat-315920.redistricting_data.TX_2020_census',
 'elections': 'cmat-315920.redistricting_data.TX_2020_elections',
 'countries': 'cmat-315920.redistricting_data.countries',
 'proposals': 'cmat-315920.redistricting_data.TX_2020_proposals',
 'tabblock': 'cmat-315920.redistricting_data.TX_2020_tabblock',
 'bg': 'cmat-315920.redistricting_data.TX_2020_bg',
 'tract': 'cmat-315920.redistricting_data.TX_2020_tract',
 'cntyvtd': 'cmat-315920.redistricting_data.TX_2020_cntyvtd',
 'cnty': 'cmat-315920.redistricting_data.TX_2020_cnty',
 'proposal': 'cmat-315920.TX_2020_sldu_plans2168.proposal',
 'nodes': 'cmat-315920.TX_2020_sldu_plans2168.cntyvtd_0_nodes'}

In [2]:
try:
    import pandas_bokeh
except:
    os.system('pip install --upgrade pandas-bokeh')
    import pandas_bokeh
pandas_bokeh.output_notebook()

plan_cols = ['geoid', 'plan', 'district']
query = f"select {', '.join(plan_cols)} from {M.output}"
plans = run_query(query).pivot(index='geoid', columns='plan', values='district')
repl = {c:str(c) for c in plans.columns}
plans.rename(columns=repl, inplace=True)

node_cols = ['geoid', 'polygon', 'county', 'total_pop', 'aland', 'density', 'polsby_popper']
query = f"select {', '.join(node_cols)} from {M.nodes}"
nodes = run_query(query).set_index('geoid')
geo = gpd.GeoSeries.from_wkt(nodes['polygon'], crs=crs_census).simplify(0.003).buffer(0) #<-- little white space @ .001 ~5.7 mb, minimal at .0001 ~10mb, with no white space ~37mb

Loading BokehJS ...

In [5]:
recolor_seed = 10
rng = np.random.default_rng(recolor_seed)
recolor = dict(zip(M.districts, rng.permutation(M.districts)))
gdf = gpd.GeoDataFrame(nodes
                       .drop(columns='polygon')
                       .join(plans.replace(recolor), how='inner')
                       , geometry=geo).reset_index()

In [ ]:
width = 600
xlim = [-106.2, -94.0]
ylim = [ 25.4 ,  36.6]
aspect = (ylim[1] - ylim[0]) / (xlim[1] - xlim[0])
height = round(aspect * width)

def f(cmap):
    fn = str(M.gpickle)
    fn = f'{fn[:fn.rfind("_")]}_{M.random_seed}_animation_{cmap}.html'
    print(f'cmap={cmap} ... saving animation to {fn}')
    fig = gdf.plot_bokeh(
        colormap = cmap,
        hovertool_string = '@geoid, @county<br>pop=@total_pop<br>density=@density{0.0}<br>land=@aland{0.0}<br>pp=@polsby_popper{0.0}',
        fill_alpha = 0.5,
        line_alpha = 0.05,
        return_html = True,
        show_figure = True,
        slider = list(repl.values()),
        slider_name = "PLAN",
        show_colorbar = False,
        xlim = xlim,
        ylim = ylim,
        figsize = (width, height),
    )
    with open(fn, 'w') as file:
        file.write(fig)
    return fig

f('Paired')

# for cmap in ['Accent', 'Blues', 'Bokeh', 'BrBG', 'BuGn', 'BuPu', 'Category10', 'Category20', 'Category20b', 'Category20c',
#              'Cividis', 'Colorblind', 'Dark2', 'GnBu', 'Greens', 'Greys', 'Inferno', 'Magma', 'OrRd', 'Oranges', 'PRGn',
#              'Paired', 'Pastel1', 'Pastel2', 'PiYG', 'Plasma', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy',
#              'RdPu', 'RdYlBu', 'RdYlGn', 'Reds', 'Set1', 'Set2', 'Set3', 'Spectral', 'Turbo', 'Viridis', 'YlGn', 'YlGnBu',
#              'YlOrBr', 'YlOrRd']:

#     f(cmap)

In [93]:
# P.sort_values('seats', ascending=False).to_csv('/home/jupyter/hi.csv')
# P[P['seats']>0.1].sort_values().to_csv('/home/jupyter/hi.csv')
A = P['seats']
B = A[A>0.1].sort_values(ascending=False)
B.to_frame()
# .to_csv('/home/jupyter/hi.csv')

,seats
Harris,5.032182
Dallas,2.779835
Tarrant,2.244938
Bexar,2.137175
Travis,1.372281
Collin,1.132196
Denton,0.964097
Hidalgo,0.926188
El Paso,0.920738
Fort Bend,0.875131


In [101]:
P = pd.DataFrame({x:d for x, d in M.adj.nodes(data=True) if x in M.counties}).T[['seats']]
S = P.copy()
for c in range(11):
    S[f'contract_{c}'] = S['seats'] < c /10
S = S.drop(columns='seats')
Q = pd.DataFrame()
Q['not contracted'] = (~S).sum (axis=0)
Q['contracted'] = S.sum (axis=0)
Q['contracted_pct']   = np.round(S.mean(axis=0) * 100).astype(int)

# Q = P.sum(axis=0).to_frame()
# Q['pct']=
Q
# P.mean(axis=0)*100
# P

,not contracted,contracted,contracted_pct
contract_0,254,0,0
contract_1,42,212,83
contract_2,25,229,90
contract_3,18,236,93
contract_4,13,241,95
contract_5,12,242,95
contract_6,12,242,95
contract_7,10,244,96
contract_8,10,244,96
contract_9,9,245,96


In [30]:
county_pop = {x:d for x, d in M.adj.nodes(data='seats') if x in M.counties}
P = list(county_pop.values())
x = np.linspace(0,1,11)
np.less.outer(P, x).shape
# {pop:(county_pop.values()<pop).sum()*100 for pop in 

(254, 11)

In [24]:
[d for x, d in M.adj.nodes(data='seats') if x in M.counties]np.sort([d for x, d in M.adj.nodes(data='seats') if x in M.counties])
np.histogram([d for x, d in M.adj.nodes(data='seats') if x in M.counties], np.linspace(0,1,11))

(array([212,  17,   7,   5,   1,   0,   2,   0,   1,   3]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))

In [18]:
from statsmodels.distributions.empirical_distribution import ECDF
# pd.DataFrame({x:d for x, d in M.adj.nodes(data=True) if x in M.counties}).T['seats'].cumsum()
([d for x, d in M.adj.nodes(data='seats') if x in M.counties])

In [20]:
e.y

array([0.        , 0.00393701, 0.00787402, 0.01181102, 0.01574803,
       0.01968504, 0.02362205, 0.02755906, 0.03149606, 0.03543307,
       0.03937008, 0.04330709, 0.04724409, 0.0511811 , 0.05511811,
       0.05905512, 0.06299213, 0.06692913, 0.07086614, 0.07480315,
       0.07874016, 0.08267717, 0.08661417, 0.09055118, 0.09448819,
       0.0984252 , 0.1023622 , 0.10629921, 0.11023622, 0.11417323,
       0.11811024, 0.12204724, 0.12598425, 0.12992126, 0.13385827,
       0.13779528, 0.14173228, 0.14566929, 0.1496063 , 0.15354331,
       0.15748031, 0.16141732, 0.16535433, 0.16929134, 0.17322835,
       0.17716535, 0.18110236, 0.18503937, 0.18897638, 0.19291339,
       0.19685039, 0.2007874 , 0.20472441, 0.20866142, 0.21259843,
       0.21653543, 0.22047244, 0.22440945, 0.22834646, 0.23228346,
       0.23622047, 0.24015748, 0.24409449, 0.2480315 , 0.2519685 ,
       0.25590551, 0.25984252, 0.26377953, 0.26771654, 0.27165354,
       0.27559055, 0.27952756, 0.28346457, 0.28740157, 0.29133